In [1]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers import LSTM, CuDNNLSTM
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint
import keras.metrics
import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
sequence_length = 15 # timestep

In [3]:
path_save = "F:\Master Project\Dataset\Extract_Data"
f_x = open(path_save+"/train_x.pickle",'rb')
f_y = open(path_save+"/train_y.pickle",'rb')
origin_train_x = pickle.load(f_x)
train_y = pickle.load(f_y)

f_x = open(path_save+"/test_x.pickle",'rb')
f_y = open(path_save+"/test_y.pickle",'rb')
origin_test_x = pickle.load(f_x)
test_y = pickle.load(f_y)

In [4]:
print(origin_train_x[0].shape)
print(len(origin_train_x))

(66330, 75)
7


### Sampling Function

In [ ]:
def sampling_x(x, is_training):
    
    frames = x
    
    random_sample_range = 3
    # Start index and sample interval for the test set
    start_i, sample_interval = 0, random_sample_range #len(frames) // sequence_length
    if is_training:
        # Randomly choose sample interval and start frame
        sample_interval = np.random.randint(1, random_sample_range + 1)
        start_i = np.random.randint(0, len(frames) - sample_interval * sequence_length + 1)

    # Extract frames as tensors
    image_sequence = []
#     for i in range(start_i, len(frames), sample_interval):
    end_i = sample_interval * sequence_length + start_i
    for i in range(start_i, end_i, sample_interval)
        # image_path = frames[i]
        if len(image_sequence) < sequence_length:
            image_sequence.append(frames[i])
        else:
            break
    image_sequence = np.array(image_sequence)   
    return image_sequence


def reform_to_sequence(data_x, is_training):
    
    train_x = np.zeros((len(data_x), sequence_length, data_x[0].shape[-1]) ) #(len,timestep, 28)
    for i,x in enumerate(data_x):
        sequence = sampling_x(x, is_training)
        train_x[i] = sequence       
        
    return train_x     


### Model Define

In [ ]:
number_feature = origin_train_x[0].shape[-1] # last index
model = Sequential()
model.add(CuDNNLSTM(50, input_shape=(sequence_length, number_feature),return_sequences=True))
model.add(Dropout(0.6))#使用Dropout函数可以使模型有更多的机会学习到多种独立的表征
model.add(CuDNNLSTM(50, input_shape=(sequence_length, number_feature)) )
model.add(Dropout(0.6))
model.add(Dense(4, activation='softmax'))
sgd = optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,)
callbacks_list = [checkpoint]

### Prepare Test Set

In [ ]:
test_x = reform_to_sequence(origin_test_x, is_training=False)

### Train

In [ ]:
num_epoch = 100

for i_ep in range(num_epoch):
    
    print('epoch: ', i_ep)
    train_x = reform_to_sequence(origin_train_x, is_training=True)
    model.fit(train_x, train_y, epochs=1,
             validation_data=(test_x,test_y), callbacks=callbacks_list)
    print("-----------------------")

